## Replicated version of CSL_create_targets that uses scaled/normalized data

In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, HTML

In [2]:
pd.options.display.max_columns = 30
# Note: Name currently assumes updated symbolic link
df = pd.read_csv('../data/csl/MFM_CSL_d6_fields_NORM.csv', index_col=0)
print(df.shape)
display(df.head())

(185413, 210)


,trans_loss,transfus_yes,transfus_all,transfus_hyster,MomNearMiss,Hysterectomy,Momdeath,MomICU,Postbleed,high_MomLOS,Bloodproduct,Posttransfus,EBLoss,high_EBLoss,hemorrhage,...,UnspecHBP,Urupture,uscar,version9,vertex,Activeherpes,iufd9,R_preg,high_BMI,high_Age,high_Gravidity,high_height,intra_abruptio,intra_previa,pre_PROM
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,0,0,0,0,0,0,0,3,0,0,-0.9992,0,0,...,0,0,1,0,9,0,0,1,0,0,2,2,0,0,0
41-00003,0,0,0,0,0,0,0,0,0,2,0,0,-0.9994,0,0,...,0,0,0,0,1,0,0,1,0,0,3,2,0,0,0
41-00004,0,0,0,0,0,0,0,0,0,1,0,0,-0.9994,0,0,...,0,0,0,0,1,0,0,1,0,0,4,3,0,0,0
41-00005,1,1,1,1,0,0,0,0,0,2,0,1,-0.9992,0,1,...,0,0,0,0,1,0,0,1,0,0,3,2,0,0,0
41-00006,0,0,0,0,0,0,0,0,0,2,0,0,-0.9990,0,0,...,0,0,0,0,1,0,0,1,0,0,1,3,0,0,0


In [3]:
# Drop all extra targets and collinear data fields
drop_list = ['transfus_all','transfus_hyster',\
             'MomNearMiss','Hysterectomy','Momdeath','MomICU','Postbleed',\
             'high_MomLOS','Bloodproduct','Posttransfus','EBLoss','high_EBLoss',\
             'hemorrhage','postpartum_hemorrhage9']
df.drop(drop_list, axis=1, inplace=True)
display(df.head())

,trans_loss,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,Admreason,AdmSBP,Alcohol,Analgesia,Anteabruption,...,UnspecHBP,Urupture,uscar,version9,vertex,Activeherpes,iufd9,R_preg,high_BMI,high_Age,high_Gravidity,high_height,intra_abruptio,intra_previa,pre_PROM
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,0,8,8,8,1.0000,-0.2759,-1.0,77,1,-0.0135,0,7,0,...,0,0,1,0,9,0,0,1,0,0,2,2,0,0,0
41-00003,0,0,0,6,3,2,-0.9414,-0.3448,0.4,1,3,-0.4170,0,8,0,...,0,0,0,0,1,0,0,1,0,0,3,2,0,0,0
41-00004,0,0,0,5,3,2,-0.9333,-0.4828,0.4,1,3,-0.2646,0,8,0,...,0,0,0,0,1,0,0,1,0,0,4,3,0,0,0
41-00005,1,1,0,11,2,3,-0.9192,-0.3448,0.6,77,4,-0.1211,0,8,0,...,0,0,0,0,1,0,0,1,0,0,3,2,0,0,0
41-00006,0,0,0,11,1,3,-0.8990,-0.1034,1.0,77,4,-0.1570,0,8,0,...,0,0,0,0,1,0,0,1,0,0,1,3,0,0,0


## Break up file into groups of columns used for different versions of output files 

In [4]:
# Save trans_loss and tranfus_yes for later
tl_series = df['trans_loss']
ty_series = df['transfus_yes']

In [5]:
# Read in variables categorized by "timing" (pre, intra, post, etc)
varTimingPath = '../data/csl/V3_d5_code_timing.csv'
var_times_df = pd.read_csv(varTimingPath, delimiter='\t')
var_times_df

,Code,timing
0,delivery,drop
1,fever9,drop
2,ga,drop
3,Lac_Unkn,drop
4,onepregflag,index
...,...,...
245,MomID,target
246,Postbleed,target
247,postpartum_hemorrhage9,target
248,Posttransfus,target


In [6]:
# NOTE: Currently don't need to worry about continuous columns since they are all "Intra"
#continuous_col = ['Admcontract', 'AdmDBP', 'Admefface', 'AdmSBP', 'BESTGA']
#CR_df = df[continuous_col]
#display(CR_df.head())
#CN_df = df_norm[continuous_col]
#display(CN_df.head())

In [7]:
pre_df = df[var_times_df[var_times_df['timing'] == 'pre']['Code'].to_list()]
display(pre_df)
intra_df = df[var_times_df[var_times_df['timing'] == 'intra']['Code'].to_list()]
display(intra_df)

,Activeherpes,Admreason,Alcohol,Anteabruption,Anteanemia,Anteasthma,Antebleed3,AnteCHBP,Antechorio,Antefetaldth,Antefetdistress,AnteGBS,AnteGDM,Antehospital,AnteLGA,...,Preeclampsia,prelaborCD,prim_hypo,R_preg,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,TTTwin,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
41-00003,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00004,0,3,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00005,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00006,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
52-07248,0,2,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
52-07249,0,5,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0


,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,chorio,...,Meconium,Momseizure,MthInd_AROM,MthInd_Oxy,Operative,Presentdel,Prolapse,ROM,ROMmeth,SE_pre,Shoulder,spontlabor,TrialLabor,Urupture,vertex
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,8,8,8,1.0000,-0.2759,-1.0,77,-0.0135,7,1,0.3818,0,0,0,...,1,0,0,0,0,88,0,0,1,0,0,1,0,0,9
41-00003,0,6,3,2,-0.9414,-0.3448,0.4,1,-0.4170,8,88,0.4545,0,0,0,...,1,0,1,1,0,1,0,0,1,0,0,0,1,0,1
41-00004,0,5,3,2,-0.9333,-0.4828,0.4,1,-0.2646,8,88,0.4636,0,0,0,...,1,0,1,1,0,1,0,0,1,0,0,0,1,0,1
41-00005,0,11,2,3,-0.9192,-0.3448,0.6,77,-0.1211,8,88,0.4000,0,0,0,...,4,0,0,0,0,1,0,0,2,0,0,1,1,0,1
41-00006,0,11,1,3,-0.8990,-0.1034,1.0,77,-0.1570,8,88,0.5091,0,0,0,...,1,0,0,0,0,1,0,0,2,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,0,8,8,8,0.5556,-0.3793,0.4,1,-0.2430,5,4,0.2818,0,0,0,...,5,0,0,1,1,1,0,0,1,0,1,0,1,0,1
52-07248,0,8,8,8,1.0000,-0.3793,-1.0,2,-0.2430,5,0,-0.8545,1,1,0,...,1,0,0,1,0,2,0,0,8,0,0,0,1,0,0
52-07249,0,8,8,8,0.3333,-0.3793,-1.0,77,-0.2430,9,0,-0.4818,0,0,0,...,1,0,0,0,0,88,0,0,2,0,0,0,0,0,9


In [8]:
ty_PI = pd.concat([intra_df,pre_df], axis=1)
ty_PI.insert(0, 'transfus_yes', ty_series)
print(ty_PI.shape)
ty_PI.head()

(185413, 195)


,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,...,Preeclampsia,prelaborCD,prim_hypo,R_preg,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,TTTwin,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,8,8,8,1.0000,-0.2759,-1.0,77,-0.0135,7,1,0.3818,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
41-00003,0,0,6,3,2,-0.9414,-0.3448,0.4,1,-0.4170,8,88,0.4545,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00004,0,0,5,3,2,-0.9333,-0.4828,0.4,1,-0.2646,8,88,0.4636,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00005,1,0,11,2,3,-0.9192,-0.3448,0.6,77,-0.1211,8,88,0.4000,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00006,0,0,11,1,3,-0.8990,-0.1034,1.0,77,-0.1570,8,88,0.5091,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [9]:
# NOTE: again, no need to create this, currently
#ty_Pre = pd.concat([pre_df], axis=1)
#ty_Pre.insert(0, 'transfus_yes', ty_series)
#print(ty_Pre.shape)
#ty_Pre.head()

In [10]:
tl_PI = pd.concat([intra_df,pre_df], axis=1)
tl_PI.insert(0, 'trans_loss', tl_series)
print(tl_PI.shape)
tl_PI.head()

(185413, 195)


,trans_loss,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,...,Preeclampsia,prelaborCD,prim_hypo,R_preg,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,TTTwin,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,8,8,8,1.0000,-0.2759,-1.0,77,-0.0135,7,1,0.3818,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
41-00003,0,0,6,3,2,-0.9414,-0.3448,0.4,1,-0.4170,8,88,0.4545,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00004,0,0,5,3,2,-0.9333,-0.4828,0.4,1,-0.2646,8,88,0.4636,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00005,1,0,11,2,3,-0.9192,-0.3448,0.6,77,-0.1211,8,88,0.4000,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
41-00006,0,0,11,1,3,-0.8990,-0.1034,1.0,77,-0.1570,8,88,0.5091,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [11]:
# NOTE: we don't need to save off a normalized version of "pre" since it has no continous variables
#tl_Pre = pd.concat([pre_df], axis=1)
#tl_Pre.insert(0, 'trans_loss', tl_series)
#print(tl_Pre.shape)
#tl_Pre.head()

### Use "overwrite" variable to control whether files are written or not
### @todo - Calculate current date and create new one every time.

In [12]:
overwrite = True

In [17]:
# Write out transfus_yes for "pre_intra"
filename = '../data/csl/MFM_CSL_d6_transfus_yes_Pre-Intra_NORM_2021-03-29.csv'
rel_filename = './MFM_CSL_d6_transfus_yes_Pre-Intra_NORM_2021-03-29.csv'
linkname = '../data/csl/CSL_ty_PI_norm.csv'
if overwrite:
    ty_PI.to_csv(filename, header=True)
    os.remove(linkname)
    os.symlink(rel_filename, linkname)
else:
    print(f'File not written: {filename}')

In [14]:
# NOTE: Not needed
# Write out transfus_yes for "pre" only
#filename = '../data/csl/CSL_d6_transfus_yes_Pre_2021-03-26.csv'
#if overwrite:
#    ty_Pre.to_csv(filename, header=True)
#else:
#    print(f'File not written: {filename}')

In [18]:
# Write out trans_loss for "pre_intra"
filename = '../data/csl/MFM_CSL_d6_trans_loss_Pre-Intra_NORM_2021-03-29.csv'
rel_filename = './MFM_CSL_d6_trans_loss_Pre-Intra_NORM_2021-03-29.csv'
linkname = '../data/csl/CSL_tl_PI_norm.csv'
if overwrite:
    tl_PI.to_csv(filename, header=True)
    os.remove(linkname)
    os.symlink(rel_filename, linkname)
else:
    print(f'File not written: {filename}')

In [16]:
# NOTE: Not needed
# Write out trans_loss for "pre" only
#filename = '../data/csl/CSL_d6_trans_loss_Pre_2021-03-26.csv'
#if overwrite:
#    tl_Pre.to_csv(filename, header=True)
#else:
#    print(f'File not written: {filename}')